## [Build a Chatbot](https://python.langchain.com/docs/tutorials/chatbot/)

1. This guide assumes familiarity with the following concepts: Chat Models, Prompt Templates and Chat History
2. As of the v0.3 release of LangChain, we recommend that LangChain users take advantage of **LangGraph persistence** to incorporate memory into new LangChain applications.


## [Overview](https://python.langchain.com/docs/tutorials/chatbot/#overview)

1. This chatbot will be able to have a **conversation and remember previous interactions.**
2. only use the language model to have a conversation. There are several other related concepts that you may be looking for:
   1. **Conversational RAG**: Enable a chatbot experience over an external source of data
   2. **Agents**: Build a chatbot that can take actions


## [Setup](https://python.langchain.com/docs/tutorials/chatbot/#setup)

```bash
pip install langchain-core langgraph>0.2.27
```

```bash
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```


In [16]:
# if in a notebook you can set them with:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [17]:
# using load_dotenv()
import os 
from dotenv import load_dotenv
load_dotenv()
# get the value of the environment variable
# print(os.getenv('OPENAI_API_KEY'))

True

## [QuickStart](https://python.langchain.com/docs/tutorials/chatbot/#quickstart)

1.  let's learn how to use a language model by itself.

```bash
pip install -qU langchain-openai
```


In [18]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo")
print(model)

client=<openai.resources.chat.completions.Completions object at 0x000002445278BD40> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002445278CDD0> root_client=<openai.OpenAI object at 0x00000244516D9850> root_async_client=<openai.AsyncOpenAI object at 0x00000244512A54C0> model_kwargs={} openai_api_key=SecretStr('**********')


1.  The model on its own does not have any concept of state. For example, if you ask a followup question:
2.  To get around this, we need to pass the entire conversation history into the model


In [19]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hello, I'm Shirshak")])
model.invoke([HumanMessage(content="What is my name")])

AIMessage(content="I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-00df054b-ed2b-462e-9886-e38c16c376e3-0', usage_metadata={'input_tokens': 11, 'output_tokens': 25, 'total_tokens': 36})

In [20]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi! I'm shirshak"),
        AIMessage(content="Hello Shirshak! How can i assist you today?"),
        HumanMessage(content="What's my name?")
    ]
)

AIMessage(content='Your name is Shirshak.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 40, 'total_tokens': 47, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f1b3e233-c010-44e2-97c3-8a0fe5e7d070-0', usage_metadata={'input_tokens': 40, 'output_tokens': 7, 'total_tokens': 47})

## [Message Persistence](https://python.langchain.com/docs/tutorials/chatbot/#message-persistence)

1. **LangGraph** implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.
2. Wrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications.


In [21]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that call the model
def call_model(state:MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START,"model")
workflow.add_node("model",call_model)

# Add memory
memory = MemorySaver()
app=workflow.compile(checkpointer=memory)


In [27]:
config = {"configurable": {"thread_id": "abc123"}}
query="Hi! I'm Shirshak"
input_messages= [HumanMessage(query)]
output = app.invoke({"messages":input_messages},config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello Shirshak! Nice to meet you. How can I help you today?


In [28]:
query = "what's my name?"

input_messages= [HumanMessage(query)]
output= app.invoke({"messages":input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Shirshak. Is there anything else you would like to know or talk about?


Great! Our chatbot now remembers things about us. If we change the config to reference a **different thread_id**, we can see that it starts the conversation fresh.


In [30]:
input_messages=[HumanMessage(query)]   
output = app.invoke({"messages":input_messages},{"configurable":{"thread_id":"newOne"}})


{'messages': [HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}, id='0269330e-9b4c-4eea-a235-43258ef67f66'),
  AIMessage(content='Sorry, I do not have the ability to know your name unless you explicitly tell me.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 12, 'total_tokens': 30, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-295a5d67-dfc6-4785-9137-64de8f1db595-0', usage_metadata={'input_tokens': 12, 'output_tokens': 18, 'total_tokens': 30})]}

In [33]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Sorry, I do not have the ability to know your name unless you explicitly tell me.


we've done is add a simple persistence layer around the model.We can start to make the more complicated and personalized by adding in a prompt template.


[Prompt templates](https://python.langchain.com/docs/tutorials/chatbot/#prompt-templates)

1. Prompt Templates help to turn raw user information into a format that the LLM can work with.


In [34]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages") 
    ]
)

In [35]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": response}

workflow.add_edge(START,"model")
workflow.add_node("model",call_model)

memory= MemorySaver()
app=workflow.compile(checkpointer=memory)

In [36]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

Our application now has two parameters-- the input messages and language. We should update our application's state to reflect this:


In [37]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [38]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Shirshak."
language = "Nepali"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

नमस्ते, शिर्षक जी। तपाईंलाई कसरी मद्दत गर्न सक्छु?


[Managing Conversation History](https://python.langchain.com/docs/tutorials/chatbot/#managing-conversation-history)

1. LangChain comes with a few built-in helpers for managing a **list of messages**. In this case we'll use the **trim_messages** helper to reduce how many messages we're sending to the model.
2. The trimmer allows us to specify **how many tokens** we want to keep, along with other parameters like if we want to always **keep the system message** and whether to allow **partial messages:**


In [39]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [41]:
workflow=StateGraph(state_schema=State)

def call_model(state:State):
    chain = prompt |model
    trimmed_messages= trimmer.invoke(state["messages"])

    response = chain.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [42]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to your personal information. How can I assist you today?


## [Streaming](https://python.langchain.com/docs/tutorials/chatbot/#streaming)

1.
